In [1]:
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px
from auxiliaries import *

In [2]:
berlin_df, berlin_nodes = read_in_network("berlin", "combined")
helsinki_df, helsinki_nodes = read_in_network("helsinki", "combined")

In [13]:
berlin_nodes

stop_I        lat        lon  \
0        105  52.528318  13.320260   
1        106  52.527903  13.323637   
2        107  52.529103  13.315981   
3        108  52.525756  13.309840   
4        109  52.525797  13.314261   
...      ...        ...        ...   
4596   10938  52.769962  13.454593   
4597   10939  52.611806  13.594948   
4598   10940  52.606930  13.601930   
4599   10946  52.734171  13.666572   
4600   12674  52.397446  13.530343   

                                               name  
0                     Wiebestr./Huttenstr. (Berlin)  
1                             Reuchlinstr. (Berlin)  
2                               Neues Ufer (Berlin)  
3                         Ilsenburger Str. (Berlin)  
4                           Goslarer Platz (Berlin)  
...                                             ...  
4596                      Stolzenhagen, Stolzenfels  
4597                        Blumberg (BAR), Gutshof  
4598                     Blumberg (BAR), Liebigstr.  
4599                               Danewitz, Kirche  
4600  Schönefeld (bei Berlin), Bohnsdorfer Chaussee  

[4601 rows x 4 columns]

In [3]:
berlin = convert_to_graph(berlin_df)
helsinki = convert_to_graph(helsinki_df)

## Connectivity

In [4]:
# For undirected graph
print(nx.is_connected(berlin))
print(nx.is_connected(helsinki))

False
False


In [5]:
print(nx.number_connected_components(berlin))
print(nx.number_connected_components(helsinki))

4
9


### Seperating connected components

In [6]:
def get_components(city):
    components = [city.subgraph(c).copy() for c in nx.connected_components(city)]
    for idx, g in enumerate(components, start=1):
        print(f"Component {idx} | Number of nodes: {len(g.nodes)} | Number of edges: {len(g.edges)}\n----------------------------------------------------------------")
        
    return components[0]

In [7]:
berlin_1 = get_components(berlin)

Component 1 | Number of nodes: 4593 | Number of edges: 12070
----------------------------------------------------------------
Component 2 | Number of nodes: 4 | Number of edges: 6
----------------------------------------------------------------
Component 3 | Number of nodes: 2 | Number of edges: 2
----------------------------------------------------------------
Component 4 | Number of nodes: 2 | Number of edges: 1
----------------------------------------------------------------


Since the majority of nodes in the Berlin network are in component 1, we will consider that and drop the rest for further analysis.

In [8]:
helsinki_1 = get_components(helsinki)

Component 1 | Number of nodes: 6879 | Number of edges: 8911
----------------------------------------------------------------
Component 2 | Number of nodes: 75 | Number of edges: 119
----------------------------------------------------------------
Component 3 | Number of nodes: 19 | Number of edges: 32
----------------------------------------------------------------
Component 4 | Number of nodes: 2 | Number of edges: 2
----------------------------------------------------------------
Component 5 | Number of nodes: 2 | Number of edges: 2
----------------------------------------------------------------
Component 6 | Number of nodes: 2 | Number of edges: 1
----------------------------------------------------------------
Component 7 | Number of nodes: 2 | Number of edges: 2
----------------------------------------------------------------
Component 8 | Number of nodes: 2 | Number of edges: 1
----------------------------------------------------------------
Component 9 | Number of nodes: 3 | Nu

Since the majority of nodes in the Helsinki network are in component 1, we will consider that and drop the rest for further analysis.

In [9]:
print(nx.is_connected(berlin_1))
print(nx.is_connected(helsinki_1))

True
True


## Robustness
The ability of a network to maintain its general structural properties (like connectivity) when it faces disruptions or attacks (loses nodes or edges).

In [10]:
# Minimum number of nodes that need to be removed to disconnect the graph
print(nx.node_connectivity(berlin_1))
print(nx.node_connectivity(helsinki_1))

1
1


In [14]:
# Which nodes?
berlin_cut_node = nx.minimum_node_cut(berlin_1)
print(berlin_cut_node)

For Berlin:

Remove Node: {7334} | Station: Series([], Name: name, dtype: object)


In [31]:
berlin_cut_node_name = berlin_nodes.loc[berlin_nodes['stop_I'] == 7334]['name'].item()
print("For Berlin:\n")
print(f"Remove Node: {berlin_cut_node} | Station: {berlin_cut_node_name}")

For Berlin:

Remove Node: {7334} | Station: Schönefeld (bei Berlin), Wehrmathen


In [34]:
# Which nodes?
helsinki_cut_node = nx.minimum_node_cut(helsinki_1)
print(helsinki_cut_node)

{7523}


In [35]:
helsinki_cut_node_name = helsinki_nodes.loc[helsinki_nodes['stop_I'] == 7334]['name'].item()
print("For helsinki:\n")
print(f"Remove Node: {helsinki_cut_node} | Station: {helsinki_cut_node_name}")

For helsinki:

Remove Node: {7523} | Station: Talman koulu


In [11]:
# Minimum number of edges that need to be removed to disconnect the graph
print(nx.edge_connectivity(berlin_1))
print(nx.edge_connectivity(helsinki_1))

1
1


In [36]:
# Which edges?
berlin_cut_edge = nx.minimum_edge_cut(berlin_1)
print(berlin_cut_edge)
helsinki_cut_edge = nx.minimum_edge_cut(helsinki_1)
print(helsinki_cut_edge)

{(8147, 8151)}
{(7441, 7523)}


In [40]:
berlin_cut_edge_name = (berlin_nodes.loc[berlin_nodes['stop_I'] == 8147, 'name'].item(),
                        berlin_nodes.loc[berlin_nodes['stop_I'] == 8151, 'name'].item())
print("For Berlin:\n")
print(f"Remove edge: {berlin_cut_edge} | Stations: {berlin_cut_edge_name}")

For Berlin:

Remove edge: {(8147, 8151)} | Stations: ('Erkner, Friedhof', 'Erkner, Jägerstr.')


In [42]:
helsinki_cut_edge_name = (helsinki_nodes.loc[helsinki_nodes['stop_I'] == 7441, 'name'].item(), 
                          helsinki_nodes.loc[helsinki_nodes['stop_I'] == 7523, 'name'].item())
print("For Helsinki:\n")
print(f"Remove edge: {helsinki_cut_edge} | Stations: {helsinki_cut_edge_name}")

For Helsinki:

Remove edge: {(7441, 7523)} | Stations: ('Mehuasema', 'Sommarnäsintie')


Robust networks have large minimum node and edge connectivity.

## Paths from one node to another

In [ ]:
# All path options from source to sink
sorted(nx.all_simple_paths(G, 111, 132))

In [ ]:
# Number of nodes to remove to disrupt path from source to sink
print(nx.node_connectivity(G, 111, 132))

# Which nodes?
print(nx.minimum_node_cut(G, 111, 132))

In [ ]:
# Removing one of the nodes
G.remove_node(389)

In [ ]:
sorted(nx.all_simple_paths(G, 111, 132))

In [ ]:
# Number of edges to remove to disrupt path from source to sink
print(nx.edge_connectivity(G, 111, 132))

# Which edges?
print(nx.minimum_edge_cut(G, 111, 132))

## Local Clustering Coefficient
Fraction of pairs of a node’s connections that are connected with each other.
The LCC of a node of degree less than 2 is 0.

In [ ]:
# LCC of node 2562
nx.clustering(G, 111)

## Global clustering coefficient

### Approach 1: 
Average the local clustering coefficient over all nodes in a graph

In [ ]:
nx.average_clustering(G)

### Approach 2: 
Percentage of ‘open triads’ that are triangles in a network.

Transitivity = (3*number of triangles)/number of open triads

In [ ]:
nx.transitivity(G)

## Diatance between two nodes

The length of the shortest path between them.

In [ ]:
# Shortest path between 111 and 2087
nx.shortest_path(G, 111, 2087)

In [ ]:
# Length of the shortest path between 111 and 2087
nx.shortest_path_length(G, 111, 2087)

## Distance from a node to all the other nodes
### 1. Breadth First Search: 
An algorithm for searching a tree data structure for a node that satisfies a given property. It starts at the tree root and explores all nodes at the present depth prior to moving on to the nodes at the next depth level.

In [ ]:
# Returns the tree graph of nodes discovered in BFS
T = nx.bfs_tree(G, 2562)
print(nx.shortest_path_length(T, 2562))
nx.draw_networkx(T, with_labels=True, node_size=100, node_color='red', width=2, font_size=10)

## Distance Measures

### 1. Average distance
between all pairs of nodes in the graph

In [ ]:
nx.average_shortest_path_length(T) # Note: graph must be connected

### 2. Diameter
Maximum distance between any pair of nodes

In [ ]:
# x.diameter(G) # Note: Graph must be strongly connected

### 3. Eccentricity of a node
Largest distance between a node and all the pther nodes

In [ ]:
# nx.eccentricity(G) # Note: Graph must be strongly connected

### 4. Radius
It is the minimum eccentricity

In [ ]:
# nx.radius(G) # Note: Graph must be strongly connected

### 5. Periphery
The set of nodes that have eccentricity equal to diameter (nodes that are on the outskirts of a graph)

In [ ]:
# nx.periphery(G) # Note: Graph must be strongly connected

### 6. Center
The set of nodes have eccentricity equal to the radius (nodes that are in the center of the graph)

In [ ]:
# nx.center(G) # Note: Graph must be strongly connected

In [ ]:
stats = pd.read_csv('network_data/berlin/stats.csv', sep=';')
stats = stats.transpose()
stats

In [ ]:
draw_nodes_on_map(berlin_nodes)